In [15]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [16]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [17]:
# import our chat-bot intents file
import json
with open('../intents.json') as json_data:
    intents = json.load(json_data)

In [18]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

33 documents
8 classes ['ask', 'care', 'depression', 'goodbye', 'sad', 'thanks', 'welcome', 'work']
75 unique stemmed words ['!', "'m", "'s", ',', '.', 'a', 'about', 'al', 'alway', 'am', 'answ', 'anyth', 'apprecy', 'ar', 'ask', 'at', 'begin', 'bye', 'can', 'car', 'colleagu', 'do', 'don', 'dont', 'ear', 'feel', 'freedom', 'friend', 'get', 'goodby', 'got', 'gtg', 'hat', 'hav', 'hello', 'help', 'hey', 'hi', 'hurt', 'i', 'import', 'in', 'lat', 'lend', 'let', 'lik', 'me', 'much', 'my', "n't", 'nee', 'no', 'nobody', 'not', 'off', 'okay', 'oth', 'quest', 'sad', 'secret', 'see', 'shar', 'someth', 'start', 'thank', 'that', 'ther', 'they', 'to', 'when', 'why', 'with', 'work', 'yo', 'you']


In [19]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [20]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 4999  | total loss: 1.22306 | time: 0.020s
| Adam | epoch: 1000 | loss: 1.22306 - acc: 0.7276 -- iter: 32/33
Training Step: 5000  | total loss: 1.15205 | time: 0.020s
| Adam | epoch: 1000 | loss: 1.15205 - acc: 0.7173 -- iter: 33/33
--
INFO:tensorflow:C:\Users\Varsha\Documents\Python_Scripts\Therabot\Sample\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [21]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [22]:
p = bow("Nobody likes my work", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0]
['ask', 'care', 'depression', 'goodbye', 'sad', 'thanks', 'welcome', 'work']


In [23]:
print(model.predict([p]))

[[3.4500968e-05 7.0039548e-02 2.3377754e-02 2.6575242e-05 6.8546138e-03
  4.1135587e-02 6.0914283e-09 8.5853136e-01]]


In [24]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )